# <center>**Space X Falcon 9 First Stage Landing Prediction**</center>

## Exploratory Data Analysis - SQLite

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 

## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

In [1]:
import csv, sqlite3
import prettytable
import pandas as pd

In [8]:
prettytable.DEFAULT = 'DEFAULT'

connector = sqlite3.connect("my_data1.db")
cur = connector.cursor()

In [9]:
%load_ext sql
%sql sqlite:///my_data1.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", connector, if_exists="replace", index=False, method="multi")

101

We will drop the blank rows in the table

In [22]:
%sql create table SPACEXTBL as select * from SPACEXTBL where Date is not null;

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTBL already exists
[SQL: create table SPACEXTBL as select * from SPACEXTBL where Date is not null;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [32]:
# unique launch sites
unique_launch_sites = %sql SELECT DISTINCT "Launch_Site" FROM SPACEXTBL;
print([row[0] for row in unique_launch_sites])

# unique orbit
unique_orbit = %sql SELECT DISTINCT "Orbit" FROM SPACEXTBL;
print([row[0] for row in unique_orbit])


 * sqlite:///my_data1.db
Done.
['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']
 * sqlite:///my_data1.db
Done.
['LEO', 'LEO (ISS)', 'Polar LEO', 'GTO', 'HEO', 'SSO', 'MEO', 'Sub-orbital']


In [38]:
# Display 5 records where launch sites begin with the string 'CCA'
%sql SELECT * FROM SPACEXTBL WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [52]:
# Display the total payload mass carried by boosters launched by NASA (CRS)
%sql SELECT SUM(PAYLOAD_MASS__KG_) FROM SPACEXTBL WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596


In [55]:
# Display average payload mass carried by booster version F9 v1.1¶
%sql SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTBL WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2928.4


In [57]:
df['Landing_Outcome'].unique()

array(['Failure (parachute)', 'No attempt', 'Uncontrolled (ocean)',
       'Controlled (ocean)', 'Failure (drone ship)',
       'Precluded (drone ship)', 'Success (ground pad)',
       'Success (drone ship)', 'Success', 'Failure', 'No attempt '],
      dtype=object)

In [67]:
# List the date when the first succesful landing outcome in ground pad was acheived.¶
%sql SELECT MIN(Date) FROM SPACEXTBL WHERE Landing_Outcome = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


MIN(Date)
2015-12-22


In [70]:
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql SELECT Booster_Version FROM SPACEXTBL WHERE Landing_Outcome = 'Success (drone ship)' AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [74]:
df['Mission_Outcome'].value_counts()

Mission_Outcome
Success                             98
Failure (in flight)                  1
Success (payload status unclear)     1
Success                              1
Name: count, dtype: int64

In [75]:
# List the total number of successful and failure mission outcomes
num_success = %sql SELECT COUNT(*) AS 'Number of Success' FROM SPACEXTBL WHERE Mission_Outcome = 'Success' OR Mission_Outcome = 'Success (payload status unclear)';
num_failure = %sql SELECT COUNT(*) AS 'Number of Failures' FROM SPACEXTBL WHERE Mission_Outcome = 'Failure (in flight)';
print(f'{num_success}\n{num_failure}')

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.
+-------------------+
| Number of Success |
+-------------------+
|         99        |
+-------------------+
+--------------------+
| Number of Failures |
+--------------------+
|         1          |
+--------------------+


In [83]:
# List all the booster_versions that have carried the maximum payload mass
%sql SELECT Booster_Version FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [97]:
%%sql
/*
    List the records which will display the month names, failure landing_outcomes in drone ship,
    booster versions, launch_site for the months in year 2015
*/
SELECT substr(Date, 6,2) as Month, Landing_Outcome, Booster_Version, Launch_Site FROM SPACEXTBL
WHERE substr(Date, 0,5) = '2015' AND Landing_Outcome = 'Failure (drone ship)' ORDER BY Month;

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
